In [1]:
cd ..

e:\GitHub\computervision


In [2]:
import torch
import os
# import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

from modules.models_util import EfficientNet, ViT, VGG19
from modules.training_util import PytorchDataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = [
    'notebooks/efficientnet/models/2024-02-01 10-31-27/best.pth',       # Fully trained 4k, 53.10%, 52.82% without transforms
    'notebooks/efficientnet/models/2024-02-01 09-55-37/best.pth',       # Fully trained 2k, 52.74%, 58.21%
    'notebooks/efficientnet/models/2024-02-02 16-34-31/best.pth',       # Fully trained even, 50.91%, 45.99%
    'notebooks/efficientnet/models/2024-31-01 18-53-33/best.pth',       # 4k, 55.84%, 53.83%
    'notebooks/efficientnet/models/2024-31-01 19-19-52/best.pth',       # 2k, 49.82%, 52.55%
    'notebooks/vit/models/01-31-2024 15-18-02/epoch_12.pth',            # ViT, 61.86%, 55.71% 8-1-1, 55.5%, 63% cor dataset
    'notebooks/vit/models/2024-02-03 21-45-07/best.pth',                # 60.4%, 52% new dataset
    'notebooks/vit/models/2024-02-03 21-53-43/best.pth',                # 48.54%
    'notebooks/vgg19/models/2024-02-04 16-46-29/epoch_15.pth',          # 55%
    'notebooks/efficientnet/models/2024-02-05 16-41-35/epoch_15.pth'    # 45.44%, 49% in same dataset
    
]
model = EfficientNet.load(weights_path[9].replace('\\','/'))
# model = ViT.load(weights_path[5].replace('\\','/'))
# model = VGG19.load(weights_path[8].replace('\\','/'))
model.to(device)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [4]:
# dataset_dir = 'E:/Datasets/cor-splits/temp-test'
dataset_dir = 'E:/Datasets/cor-splits/sgkf-8-1-1'
# dataset_dir = 'E:/Datasets/cor'

data_transforms = EfficientNet.data_transforms
dataset = PytorchDataset(dataset_dir, data_transforms, training_phases=['test'])
dataloader = dataset.dataloaders['test']

In [5]:
from sklearn.metrics import confusion_matrix

correct_predictions = 0
total_samples = 0

n_cls = dataset.get_n_classes()

acc = [0 for _ in range(n_cls)]

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad(): # no backtracking
    for i, (inputs, labels) in enumerate(dataset.dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        
        # Append batch prediction results
        predlist=torch.cat([predlist,predicted.view(-1).cpu()])
        lbllist=torch.cat([lbllist,labels.view(-1).cpu()])
        
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print('Confusion Matrix')
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print('Accuracy per class')
print(class_accuracy)

accuracy = correct_predictions/total_samples
print(f'\nAccuracy: {accuracy * 100:.2f}%')

[[259 170]
 [238 191]]
[60.37296037 44.52214452]

Accuracy: 52.45%
